In [53]:
import numpy as np
import tensorly as tl
from tensorly.decomposition import  tensor_train
from sklearn.model_selection import train_test_split
import scipy.linalg as linalg
import os
import cv2

tl.set_backend('numpy')


In [71]:
class tt3d:
    def __init__(self):
        self.cores = None
        self.fitted = False
    
    def fit(self,tensor):
        tensor = tl.tensor(tensor)
        self.cores = tensor_train(tensor, [1, len(tensor[:,0,0]), len(tensor[0,:,0]), 1])
        #for i,core in enumerate(self.cores):
         #   print(f"index = {i} shape = {core.shape}")
    
    def predict(self,person):
        [G1, G2, G3] = self.cores
        G1 = G1[0,:,:]
        G3 = G3[:,:,0]
        z_k = np.dot(G1.T , person)
        [_,ne, _] = G2.shape
        min_e = 1e10
        min_p = 1e10
        min_DTT = 1e10
        for e in range(ne):
            G2_e = G2[:,e,:]
            Q,R = np.linalg.qr(G2_e,mode="reduced")
            alfa_e = np.dot(Q.T,z_k)
            alfa_e = linalg.solve(R, alfa_e)
            for p in range(len(G3[0,:])):
                norm = linalg.norm(alfa_e - G3[:,p],2)
                if(norm < min_DTT):
                    #print(f"e = {e} , p = {p} , norm = {linalg.norm(alfa_e - G3[:,p],2)}")
                    min_DTT = norm
                    min_e = e
                    min_p = p
    
        return(min_e,min_p)


        


In [72]:
class tt4d:
    def __init__(self):
        self.cores = None
        self.fitted = False
    
    def fit(self,tensor):
        tensor = tl.tensor(tensor)
        self.cores = tensor_train(tensor, [1, len(tensor[:,0,0,0]), len(tensor[0,0,:,0])*len(tensor[0,0,0,:]), len(tensor[0,0,0,:]), 1])
        #for i,core in enumerate(self.cores):
         #   print(f"index = {i} shape = {core.shape}")
    
    def predict(self,z):
        [G1, G2, G3, G4] = self.cores
        z = z.flatten()
        G12 = tl.tenalg.mode_dot(G2,G1[0,:,:],0)
        G4 = G4[:,:,0]
        [_,ne, _] = G3.shape
        min_e = 1e10
        min_p = 1e10
        min_DTT = 1e10
        for e in range(ne):
            C_e = tl.tenalg.mode_dot(G12, G3[:,e,:].T,2)
            alfa_e = linalg.lstsq(tl.base.unfold(C_e, 2).T, z)
            alfa_e = alfa_e[0]
            for p in range(len(G4[0,:])):
                norm = linalg.norm(alfa_e - G4[:,p],2)
                if(norm < min_DTT):
                    #print(f"e = {e} , p = {p} , norm = {linalg.norm(alfa_e - G4[:,p],2)}")
                    min_DTT = norm
                    min_e = e
                    min_p = p
    
        return(min_e,min_p)


In [56]:

def load_orl_tensor3d(dataset_path):
    num_persons = 40
    num_expressions = 10
    img_size = (92, 112)  
    img_vector_size = img_size[0] * img_size[1]  

    orl_tensor = np.zeros((img_vector_size, num_expressions, num_persons))
    labels = np.zeros((num_expressions, num_persons), dtype=int)  

    for person_id in range(1, num_persons + 1):
        person_folder = os.path.join(dataset_path, f"s{person_id}")

        for img_id in range(1, num_expressions + 1):
            img_path = os.path.join(person_folder, f"{img_id}.pgm")
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  
            
            img_vector = img.flatten()

            orl_tensor[:, img_id - 1, person_id - 1] = img_vector
            labels[img_id - 1, person_id - 1] = person_id - 1 

    return tl.tensor(orl_tensor), labels

dataset_path = r"C:\\Users\\grgos\\Downloads\\matricne\\archive"

orl_tensor3d, orl_labels = load_orl_tensor3d(dataset_path)

print("ORL Tensor Shape (Pixels, Expressions, Persons):", orl_tensor3d.shape)
print("Labels Shape:", orl_labels.shape)


ORL Tensor Shape (Pixels, Expressions, Persons): (10304, 10, 40)
Labels Shape: (10, 40)


In [57]:
temp = tt3d()
temp.fit(orl_tensor3d)
temp.predict(orl_tensor3d[:,4,25])

(4, 25)

In [58]:

def load_orl_tensor4d(dataset_path):
    num_persons = 40
    num_expressions = 10
    img_size = (92, 112)  

    orl_tensor = np.zeros((img_size[1], img_size[0], num_expressions, num_persons))
    labels = np.zeros((num_expressions, num_persons), dtype=int)  

    for person_id in range(1, num_persons + 1):
        person_folder = os.path.join(dataset_path, f"s{person_id}")

        for img_id in range(1, num_expressions + 1):
            img_path = os.path.join(person_folder, f"{img_id}.pgm")
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  
            

            orl_tensor[:,:, img_id - 1, person_id - 1] = img[:,:]
            labels[img_id - 1, person_id - 1] = person_id - 1 

    return tl.tensor(orl_tensor), labels

dataset_path = r"C:\\Users\\grgos\\Downloads\\matricne\\archive"

orl_tensor4d, orl_labels = load_orl_tensor4d(dataset_path)

print("ORL Tensor Shape (Pixels, Expressions, Persons):", orl_tensor4d.shape)
print("Labels Shape:", orl_labels.shape)



ORL Tensor Shape (Pixels, Expressions, Persons): (112, 92, 10, 40)
Labels Shape: (10, 40)


In [83]:
temp4d = tt4d()
temp4d.fit(orl_tensor4d)
temp4d.predict(orl_tensor4d[:,:,8,39])

(8, 39)

In [82]:
import random
x = list(range(0, 10))
random.shuffle(x)
print(x)
test_split =x[:2]
train_split = x[2:]
orl_tensor3d_train = orl_tensor3d[:,train_split,:]
orl_tensor3d_test = orl_tensor3d[:,test_split,:]
test3d = tt3d()
test3d.fit(orl_tensor3d_train)
num = 0
for i in range(2):
    for j in range(40):
        output = test3d.predict(orl_tensor3d_test[:,i,j])
        if output[1] == j:
            num += 1
print(num / 80)

[1, 7, 0, 4, 8, 3, 6, 5, 2, 9]
0.9


In [63]:
x = list(range(0, 10))
random.shuffle(x)
print(x)
test_split =x[:2]
train_split = x[2:]
orl_tensor4d_train = orl_tensor4d[:,:,train_split,:]
orl_tensor4d_test = orl_tensor4d[:,:,test_split,:]
test4d = tt4d()
test4d.fit(orl_tensor4d_train)
num = 0
for i in range(2):
    for j in range(40):
        output = test4d.predict(orl_tensor4d_test[:,:,i,j])
        if output[1] == j:
            num += 1
print(num / 80)

[2, 0, 6, 7, 5, 9, 1, 3, 4, 8]
0.975


In [109]:
def load_coil_tensor3d(dataset_path):
    num_persons = 20
    num_expressions = 72
    img_size = (128, 128)  
    img_vector_size = img_size[0] * img_size[1]  

    coil_tensor = np.zeros((img_vector_size, num_expressions, num_persons))

    for person_id in range(1, num_persons + 1):
        person_folder = os.path.join(dataset_path, f"{person_id}")
        #print(person_folder)

        for img_id in range(0, num_expressions):
            img_path = os.path.join(person_folder, f"obj{person_id}__{img_id}.png")
        #   print(img_path)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  
            
            img_vector = img.flatten()

            coil_tensor[:, img_id - 1, person_id - 1] = img_vector

    return tl.tensor(coil_tensor)

dataset_path = r"C:\\Users\\grgos\\Downloads\\matricne\\archive_coil20\\coil-20"

coil_tensor3d = load_coil_tensor3d(dataset_path)

print("COIL Tensor Shape (Pixels, Expressions, Persons):", coil_tensor3d.shape)

COIL Tensor Shape (Pixels, Expressions, Persons): (16384, 72, 20)


In [110]:
x = list(range(0, 72))
random.shuffle(x)
print(x)
test_split =x[:14]
train_split = x[14:]
coil_tensor3d_train = coil_tensor3d[:,train_split,:]
coil_tensor3d_test = coil_tensor3d[:,test_split,:]
test3d = tt3d()
test3d.fit(coil_tensor3d_train)
num = 0
for i in range(14):
    for j in range(20):
        output = test3d.predict(coil_tensor3d_test[:,i,j])
        if output[1] == j:
            num += 1
print(num / (20*14))

[49, 5, 34, 47, 67, 4, 50, 36, 13, 20, 10, 40, 24, 8, 11, 46, 55, 68, 27, 18, 31, 44, 1, 43, 14, 54, 70, 17, 0, 42, 65, 51, 19, 35, 21, 52, 16, 29, 71, 2, 33, 61, 6, 32, 41, 25, 45, 28, 22, 15, 57, 60, 62, 69, 39, 7, 23, 59, 30, 26, 63, 58, 12, 66, 3, 9, 48, 53, 64, 38, 56, 37]
0.9928571428571429


In [111]:
def load_coil_tensor4d(dataset_path):
    num_persons = 20
    num_expressions = 72
    img_size = (128, 128)  
    img_vector_size = img_size[0] * img_size[1]  

    coil_tensor = np.zeros((128,128, num_expressions, num_persons))

    for person_id in range(1, num_persons + 1):
        person_folder = os.path.join(dataset_path, f"{person_id}")
        #print(person_folder)

        for img_id in range(0, num_expressions):
            img_path = os.path.join(person_folder, f"obj{person_id}__{img_id}.png")
        #   print(img_path)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  
            
            #img_vector = img.flatten()

            coil_tensor[:,:, img_id - 1, person_id - 1] = img[:,:]

    return tl.tensor(coil_tensor)

dataset_path = r"C:\\Users\\grgos\\Downloads\\matricne\\archive_coil20\\coil-20"

coil_tensor4d = load_coil_tensor4d(dataset_path)

print("COIL Tensor Shape (Pixels, Expressions, Persons):", coil_tensor4d.shape)

COIL Tensor Shape (Pixels, Expressions, Persons): (128, 128, 72, 20)


In [113]:
x = list(range(0, 72))
random.shuffle(x)
print(x)
test_split =x[:14]
train_split = x[14:]
coil_tensor4d_train = coil_tensor4d[:,:,train_split,:]
coil_tensor4d_test = coil_tensor4d[:,:,test_split,:]
test4d = tt4d()
test4d.fit(coil_tensor4d_train)
num = 0
for i in range(14):
    for j in range(20):
        output = test4d.predict(coil_tensor4d_test[:,:,i,j])
        if output[1] == j:
            num += 1
print(num / (20*14))

[7, 66, 61, 68, 27, 58, 55, 57, 29, 11, 45, 0, 65, 50, 46, 12, 25, 26, 21, 8, 4, 62, 34, 71, 49, 1, 40, 5, 70, 17, 33, 6, 64, 60, 63, 2, 47, 24, 52, 56, 23, 16, 32, 67, 36, 38, 15, 3, 28, 69, 19, 48, 42, 14, 37, 30, 35, 18, 31, 54, 51, 22, 41, 43, 44, 53, 9, 39, 13, 20, 59, 10]
0.9964285714285714
